In [0]:
!pip install tensorflow==2.0

In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
tf.__version__

'2.0.0'

In [0]:
mnist = tf.keras.datasets.mnist # get mnist dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train/255.0, x_test/255.0 # normalise the data

In [0]:
x_train = x_train[..., None] # add channel dim
x_test = x_test[..., None] # add channel dim

It is incredibly helpful to go through the `tf.data` [documentation](https://www.tensorflow.org/guide/data). 

In [0]:
# use tf.data to batch and shuffle the dataset
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(60000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(100)

Recommended reading: all sections including and after [this one](https://www.tensorflow.org/guide/keras/functional#extending_the_api_by_writing_custom_layers)  
[This guide](https://www.tensorflow.org/guide/keras/custom_layers_and_models) on custom models and layers

In [0]:
class cnn_model(tf.keras.models.Model):
  def __init__ (self):
    super().__init__()
    self.conv1 = tf.keras.layers.Conv2D(32, 3, activation='relu')
    self.flatten = tf.keras.layers.Flatten()
    self.dense = tf.keras.layers.Dense(128, activation='relu')
    self.out = tf.keras.layers.Dense(10, activation='softmax')

  def call(self, input):
    x = self.conv1(input)
    flat = self.flatten(x)
    x = self.dense(flat)
    return self.out(x)

# create an instance of the model
model = cnn_model()

In [0]:
# choose optimizer and loss function
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.optimizers.Adam()

In [0]:
# select metrics to measure loss and accuracy
# these metrics will accumulate over the epoch
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_acc = tf.keras.metrics.SparseCategoricalAccuracy(name='train_acc')

test_loss =  tf.keras.metrics.Mean(name='test_loss')
test_acc = tf.keras.metrics.SparseCategoricalAccuracy(name='test_acc')

Read [this](https://www.tensorflow.org/guide/function#setup) guide on `@tf.function` decorator, and [this section](https://www.tensorflow.org/guide/keras/train_and_evaluate#using_the_gradienttape_a_first_end-to-end_example) on gradient tape.

In [0]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_acc(labels, predictions)

In [0]:
@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_acc(labels, predictions)

In [0]:
EPOCHS = 5

for epoch in range(EPOCHS):
  i = 0
  for images, labels in train_ds:
    train_step(images, labels)
    if i%1000 == 0:
      print("Epoch:{}, Loss:{}, Accuracy:{}".format(epoch+1, train_loss.result(), train_acc.result()))
    i += 1
  # reset the metrics for the next epoch
  train_loss.reset_states()
  train_acc.reset_states()


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch:1, Loss:2.3081159591674805, Accuracy:0.125
Epoch:1, Loss:0.18764735758304596, Accuracy:0.9434003233909607
Epoch:2, Loss:0.1389603614807129, Accuracy:0.9577059149742126
Epoch:2, Loss:0.10573633760213852, Accuracy:0.9676743149757385
Epoch:3, Loss:0.09119783341884613, Accuracy:0.9721240997314453
Epoch:3, Loss:0.07644958794116974, Accuracy:0.9766693711280823
Epoch:4, Loss:0.06859026849269867, Accuracy:0.9789814949035645
Epoch:4, Loss:0.060195233672857285, Accuracy:0.9814509153366089
Epoch:5, Loss:0.05

In [0]:
for images, labels in test_ds:
  test_step(images, labels)
print("Loss:{}, Accuracy:{}".format(test_loss.result(), test_acc.result()))


Loss:0.06790091842412949, Accuracy:0.9825000166893005
